# ANN project

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd
from hyperopt import STATUS_OK,fmin, tpe, hp, Trials
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

import mlflow
from mlflow.models import infer_signature


x:\nasim_xhqpjmy\Code\MLops\MLflow-3\.venv\Lib\site-packages\hyperopt\atpe.py:19: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources


In [2]:
data = pd.read_csv("https://raw.githubusercontent.com/mlflow/mlflow/master/tests/datasets/winequality-white.csv",sep=";")


In [3]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


#### spliting

In [4]:
train,test = train_test_split(data,test_size=0.2,random_state=42)


In [5]:
#train data
train_x=train.drop(["quality"], axis=1).values    #independent
train_y=train[["quality"]].values.ravel()         #dependent


#test data -for evaluation
test_x=test.drop(["quality"], axis=1).values      #independent
test_y=test[["quality"]].values.ravel()           #dependent


#futhur spliting the training data -> train and validation -cheacking performance
train_x,valid_x,train_y,valid_y = train_test_split(train_x, train_y, test_size=0.2, random_state=42)

signature = infer_signature(train_x, train_y)

### ANN model

In [7]:
def train_model(params,epochs,train_x,train_y,valid_x,valid_y,test_x,test_y):
    #define the architecture
    mean=np.mean(train_x,axis=0)
    var=np.var(train_x,axis=0)

    model = tf.keras.Sequential([
        keras.Input([train_x.shape[1]]),
        keras.layers.Normalization(mean=mean, variance=var),
        keras.layers.Dense(64,activation='relu'),                  #single hidden layer with 64 neurons
        keras.layers.Dense(1)
    ])
    #compile the model
    model.compile(optimizer=keras.optimizers.SGD(
        learning_rate=params['learning_rate'], momentum=params['momentum']
    ),loss="mean_squared_error",metrics=[keras.metrics.RootMeanSquaredError()])

    #train the model and track em
    with mlflow.start_run(nested=True):
        model.fit(train_x,train_y,validation_data=(valid_x,valid_y),epochs=epochs,batch_size=32)

        #evaluate
        eval_result = model.evaluate(valid_x, valid_y, batch_size=32)

        eval_rmse=eval_result[1]

        #log the parameter and results
        mlflow.log_params(params)
        mlflow.log_metric("eval_rmse",eval_rmse) 

        #log model
        mlflow.tensorflow.log_model(model, "model", signature=signature)

        return {"loss": eval_rmse, "status": STATUS_OK, "model": model}

In [8]:
def objective(params):
    results=train_model(
        params,
        epochs=3,
        train_x=train_x,
        train_y=train_y,
        valid_x=valid_x,
        valid_y=valid_y,
        test_x=test_x,
        test_y=test_y
    )
    return results

In [9]:
space={
    "learning_rate": hp.loguniform("learning_rate",np.log(1e-5), np.log(1e-0)), 
    "momentum": hp.uniform("momentum", 0.0, 1.0 )
}

In [10]:
mlflow.set_experiment("WINEs-Quality-!!-new")
with mlflow.start_run():
    #conduct hyperparameter tuning
    trials=Trials()
    best=fmin(
        fn=objective,
        space=space,
        algo=tpe.suggest,
        max_evals=4,
        trials=trials
    )

    best_runs=sorted(trials.results, key=lambda x: x["loss"])[0]

    #log best parameters
    mlflow.log_params(best)
    mlflow.log_metric("eval_rmse",best_runs["loss"])
    mlflow.tensorflow.log_model(best_runs["model"], "model", signature=signature)

    #print the best parameters and corresponding loss
    print(f"Best parameters: {best}")
    print(f"Best eval rmse: {best_runs['loss']}")

2025/08/23 15:12:26 INFO mlflow.tracking.fluent: Experiment with name 'WINEs-Quality-!!-new' does not exist. Creating a new experiment.


Epoch 1/3                                            

 1/98 ━━━━━━━━━━━━━━━━━━━━ 48s 496ms/step - loss: 34.8277 - root_mean_squared_error: 5.9015
34/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 18.1439 - root_mean_squared_error: 4.1771   
77/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 11.8489 - root_mean_squared_error: 3.3033
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.4881 - root_mean_squared_error: 2.1185 - val_loss: 1.8545 - val_root_mean_squared_error: 1.3618

Epoch 2/3                                            

 1/98 ━━━━━━━━━━━━━━━━━━━━ 2s 26ms/step - loss: 1.5576 - root_mean_squared_error: 1.2480
40/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.4499 - root_mean_squared_error: 1.2037 
87/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.4022 - root_mean_squared_error: 1.1838
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2967 - root_mean_squared_error: 1.1387 - val_loss: 1.4078 - val_root_mean_squared_error: 1.1865

Epoch 3/3                                            

 1/9

2025/08/23 15:12:35 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\nasim_xhqpjmy\AppData\Local\Temp\tmpo3hokngp\model, flavor: tensorflow). Fall back to return ['tensorflow==2.20.0', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 



Epoch 1/3                                                                     

 1/98 ━━━━━━━━━━━━━━━━━━━━ 37s 386ms/step - loss: 33.1694 - root_mean_squared_error: 5.7593
42/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 15.7486 - root_mean_squared_error: 3.8718   
88/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 10.5255 - root_mean_squared_error: 3.0983
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 4.2375 - root_mean_squared_error: 2.0585 - val_loss: 1.6507 - val_root_mean_squared_error: 1.2848

Epoch 2/3                                                                     

 1/98 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - loss: 1.1238 - root_mean_squared_error: 1.0601
36/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1704 - root_mean_squared_error: 1.0818 
87/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 1.1848 - root_mean_squared_error: 1.0884
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1732 - root_mean_squared_error: 1.0831 - val_loss: 1.1720 - val_root_mean_squared_error: 1.0826

Epoch 3/3

2025/08/23 15:12:41 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\nasim_xhqpjmy\AppData\Local\Temp\tmp84wuahn6\model, flavor: tensorflow). Fall back to return ['tensorflow==2.20.0', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 



...

Epoch 1/3                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 31s 324ms/step - loss: 32.6125 - root_mean_squared_error: 5.7107
29/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 29.0841 - root_mean_squared_error: 5.3897   
55/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 25.8809 - root_mean_squared_error: 5.0730
86/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 22.6477 - root_mean_squared_error: 4.7254
98/98 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - loss: 13.3892 - root_mean_squared_error: 3.6591 - val_loss: 3.4445 - val_root_mean_squared_error: 1.8559

Epoch 2/3                                                                      

 1/98 ━━━━━━━━━━━━━━━━━━━━ 2s 25ms/step - loss: 2.2529 - root_mean_squared_error: 1.5010
35/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.7644 - root_mean_squared_error: 1.6620 
73/98 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - loss: 2.6000 - root_mean_squared_error: 1.6112
98/98 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 2.2006 - root_me

2025/08/23 15:12:55 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\nasim_xhqpjmy\AppData\Local\Temp\tmpvc006umc\model, flavor: tensorflow). Fall back to return ['tensorflow==2.20.0', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 



100%|██████████| 4/4 [00:28<00:00,  7.10s/trial, best loss: 0.9478039145469666]


2025/08/23 15:13:00 WARNING mlflow.utils.environment: Encountered an unexpected error while inferring pip requirements (model URI: C:\Users\nasim_xhqpjmy\AppData\Local\Temp\tmp_ea_wph7\model, flavor: tensorflow). Fall back to return ['tensorflow==2.20.0', 'cloudpickle==3.1.1']. Set logging level to DEBUG to see the full traceback. 


Best parameters: {'learning_rate': np.float64(0.0019048307557839234), 'momentum': np.float64(0.7523666840547196)}
Best eval rmse: 0.9478039145469666


## Inferencing

In [11]:
from mlflow.models import validate_serving_input

model_uri = 'runs:/0e09fe2dec7042a983dff430b5f9d913/model'

# The logged model does not contain an input_example.
# Manually generate a serving payload to verify your model prior to deployment.
from mlflow.models import convert_input_example_to_serving_input

# Define INPUT_EXAMPLE via assignment with your own input example to the model
# A valid input example is a data instance suitable for pyfunc prediction
serving_payload = convert_input_example_to_serving_input(test_x)

# Validate the serving payload works on the model
validate_serving_input(model_uri, serving_payload)

x:\nasim_xhqpjmy\Code\MLops\MLflow-3\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


array([[ 5.341753 ],
       [ 6.9756794],
       [ 6.258979 ],
       [ 5.1023846],
       [ 5.5452633],
       [ 7.217578 ],
       [ 5.7923594],
       [ 5.296441 ],
       [ 6.314829 ],
       [ 5.46634  ],
       [ 7.2379923],
       [ 4.239955 ],
       [ 7.173662 ],
       [ 4.640109 ],
       [ 6.7528353],
       [ 5.1208506],
       [ 7.326352 ],
       [ 6.5189934],
       [ 6.304165 ],
       [ 5.192891 ],
       [ 5.2196474],
       [ 6.3846245],
       [ 4.6688414],
       [ 5.7255745],
       [ 5.535819 ],
       [ 4.848283 ],
       [ 4.6065416],
       [ 5.9691916],
       [ 5.8709135],
       [ 4.888839 ],
       [ 5.3804626],
       [ 5.087451 ],
       [ 6.4747906],
       [ 5.026763 ],
       [ 5.675788 ],
       [ 6.538595 ],
       [ 6.8305607],
       [ 5.034729 ],
       [ 5.2004232],
       [ 6.0664883],
       [ 5.660515 ],
       [ 5.0958414],
       [ 5.2418127],
       [ 5.9777126],
       [ 4.446406 ],
       [ 6.45417  ],
       [ 5.8141994],
       [ 5.64

In [12]:
model_uri = 'runs:/0e09fe2dec7042a983dff430b5f9d913/model'

loaded_model=mlflow.pyfunc.load_model(model_uri)

import pandas as pd
loaded_model.predict(pd.DataFrame(test_x))

31/31 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


array([[ 5.341753 ],
       [ 6.9756794],
       [ 6.258979 ],
       [ 5.1023846],
       [ 5.5452633],
       [ 7.217578 ],
       [ 5.7923594],
       [ 5.296441 ],
       [ 6.314829 ],
       [ 5.46634  ],
       [ 7.2379923],
       [ 4.239955 ],
       [ 7.173662 ],
       [ 4.640109 ],
       [ 6.7528353],
       [ 5.1208506],
       [ 7.326352 ],
       [ 6.5189934],
       [ 6.304165 ],
       [ 5.192891 ],
       [ 5.2196474],
       [ 6.3846245],
       [ 4.6688414],
       [ 5.7255745],
       [ 5.535819 ],
       [ 4.848283 ],
       [ 4.6065416],
       [ 5.9691916],
       [ 5.8709135],
       [ 4.888839 ],
       [ 5.3804626],
       [ 5.087451 ],
       [ 6.4747906],
       [ 5.026763 ],
       [ 5.675788 ],
       [ 6.538595 ],
       [ 6.8305607],
       [ 5.034729 ],
       [ 5.2004232],
       [ 6.0664883],
       [ 5.660515 ],
       [ 5.0958414],
       [ 5.2418127],
       [ 5.9777126],
       [ 4.446406 ],
       [ 6.45417  ],
       [ 5.8141994],
       [ 5.64

In [13]:
#registring the model
mlflow.register_model(model_uri, "WineQualityModel")

Successfully registered model 'WineQualityModel'.
Created version '1' of model 'WineQualityModel'.


<ModelVersion: aliases=[], creation_timestamp=1755942921837, current_stage='None', description=None, last_updated_timestamp=1755942921837, name='WineQualityModel', run_id='0e09fe2dec7042a983dff430b5f9d913', run_link=None, source='file:///x:/nasim_xhqpjmy/Code/MLops/MLflow-3/project-2/mlruns/663289339673427619/0e09fe2dec7042a983dff430b5f9d913/artifacts/model', status='READY', status_message=None, tags={}, user_id=None, version=1>